# March Madness Machine Learning Project 
#### never lose your family braket challenge to your aunt who knows nothing about basketball ever again 
## James Griffin, Noah Brown 
### Brigham Young University - Machine Learning Project 

In [1]:
# Importing LOTS of stuff 

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import learning_curve
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split

#from online suggestions I was reading 
from string import ascii_letters
import seaborn as sns
import statsmodels.api as sm # for linear regression, found it on some article 

# to solve problems that I am encounterinbg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt #plotting for visualization purposes of story telling 
import numpy as np

from tqdm import tqdm

import datetime
import gc
import numpy as np
import os
import operator
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
sns.set_palette(sns.color_palette('hls', 7))

from statistics import mean

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import describe
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import KFold, RepeatedKFold, GroupKFold
import random 
from random import sample

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from collections import Counter
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
!pip install imblearn
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV



### Read In data 

In [824]:
# teams that could make the tournament all D1 teams 

teams = pd.read_csv("Teams.csv")
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [825]:
# data that we created with the seeds of all teams in tournaments since 2003 ranked from 1-64. 
# we do not rank the play in teams that lose, they are not part of the field of 64 and this we do not take into 
#account the losers of the play in games.  We back up this assumption because you are not requiered to pick them in 
#the ESPN bracket challenge, which to participate, you don't have to submit until before the first round begins 
# on day 136, which is always a Thursday. 

power_seeds = pd.read_csv("PowerSeeds.csv")


power_seeds.tail()

,Season,PowerSeed,TeamName
955,2009,60,Robert Morris
956,2009,61,Chattanooga
957,2009,62,Radford
958,2009,63,ETSU
959,2009,64,Morehead St


In [826]:
# Detailed in game statistics since 2003 

tourney_deets = pd.read_csv("NCAATourneyDetailedResults.csv")
tourney_deets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [827]:
# details of the regular season - same basic structure, except they take into account who was home 

reg_deets = pd.read_csv("RegularSeasonDetailedResults.csv")
reg_deets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [828]:
# basic tournament details, who won and lost, score 

tourney_compact_deets =  pd.read_csv("NCAATourneyCompactResults.csv")
tourney_compact_deets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [829]:
# teams seeding and region since 1985 

tourney_seeds = pd.read_csv("NCAATourneySeeds.csv")
tourney_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


Ok a few obsticals right off the bat, WTeamID and LteamID are the foreign keys to our teams table. This reminds me of sql how that will be a primary key and the W/LTeamID will be the foreign key. We'll connect these through the pandas pd.merge command 

In [830]:
#getting team names for the teams that made the tournament 

tourney_teams = pd.merge(teams, tourney_deets, left_on = "TeamID", right_on = "WTeamID")

tourney_teams = tourney_teams.drop(axis = 1, columns = "FirstD1Season")
tourney_teams = tourney_teams.drop(axis = 1, columns = "LastD1Season")

tourney_teams.head()

,TeamID,TeamName,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,1104,Alabama,2004,136,1104,65,1356,64,N,0,...,22,4,5,10,20,15,10,4,1,14
1,1104,Alabama,2004,138,1104,70,1390,67,N,0,...,24,8,11,22,27,12,12,2,0,31
2,1104,Alabama,2004,143,1104,80,1393,71,N,0,...,14,14,20,16,19,12,18,5,4,20
3,1104,Alabama,2006,136,1104,90,1266,85,N,0,...,22,14,19,13,15,13,12,9,4,22
4,1107,Albany NY,2014,134,1107,71,1291,64,N,0,...,37,8,12,6,17,13,4,3,1,16


In [831]:
#for loosing team 

tourney_teams =  pd.merge(teams, tourney_teams, left_on = "TeamID", right_on = "LTeamID")

tourney_teams = tourney_teams.drop(axis = 1, columns = "FirstD1Season")
tourney_teams = tourney_teams.drop(axis = 1, columns = "LastD1Season")

tourney_teams.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y,Season,DayNum,WTeamID,WScore,LTeamID,LScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,1102,Air Force,1228,Illinois,2006,136,1228,78,1102,69,...,27,14,18,1,14,15,11,7,1,13
1,1102,Air Force,1314,North Carolina,2004,136,1314,63,1102,52,...,25,10,13,2,19,15,16,7,2,15
2,1103,Akron,1211,Gonzaga,2009,136,1211,77,1103,64,...,23,12,18,11,15,15,8,0,1,23
3,1103,Akron,1323,Notre Dame,2011,137,1323,69,1103,56,...,21,3,6,8,24,10,8,6,4,21
4,1103,Akron,1433,VA Commonwealth,2013,136,1433,88,1103,42,...,13,11,19,9,20,5,22,2,6,12


In [832]:
# Ok, so columns with subscript x  are losing team, and teams 
# with subscript y are winning teams, 

tourney_teams = tourney_teams.rename(columns = {'TeamName_x': 'LTeam', 'TeamName_y':'WTeam'})


tourney_teams.head()

,TeamID_x,LTeam,TeamID_y,WTeam,Season,DayNum,WTeamID,WScore,LTeamID,LScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,1102,Air Force,1228,Illinois,2006,136,1228,78,1102,69,...,27,14,18,1,14,15,11,7,1,13
1,1102,Air Force,1314,North Carolina,2004,136,1314,63,1102,52,...,25,10,13,2,19,15,16,7,2,15
2,1103,Akron,1211,Gonzaga,2009,136,1211,77,1103,64,...,23,12,18,11,15,15,8,0,1,23
3,1103,Akron,1323,Notre Dame,2011,137,1323,69,1103,56,...,21,3,6,8,24,10,8,6,4,21
4,1103,Akron,1433,VA Commonwealth,2013,136,1433,88,1103,42,...,13,11,19,9,20,5,22,2,6,12


In [833]:
tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamID_y")
tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamID_x")
tourney_teams.head()

,LTeam,WTeam,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,Air Force,Illinois,2006,136,1228,78,1102,69,N,0,...,27,14,18,1,14,15,11,7,1,13
1,Air Force,North Carolina,2004,136,1314,63,1102,52,N,0,...,25,10,13,2,19,15,16,7,2,15
2,Akron,Gonzaga,2009,136,1211,77,1103,64,N,0,...,23,12,18,11,15,15,8,0,1,23
3,Akron,Notre Dame,2011,137,1323,69,1103,56,N,0,...,21,3,6,8,24,10,8,6,4,21
4,Akron,VA Commonwealth,2013,136,1433,88,1103,42,N,0,...,13,11,19,9,20,5,22,2,6,12


Importing power seeds

In [834]:
tourney_teams = pd.merge(power_seeds, tourney_teams, left_on = ["TeamName", "Season"], right_on = ["WTeam", "Season"])

tourney_teams.tail()

,Season,PowerSeed,TeamName,LTeam,WTeam,DayNum,WTeamID,WScore,LTeamID,LScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
978,2009,47,Wisconsin,Florida St,Wisconsin,137,1458,61,1199,59,...,13,15,18,4,29,6,15,6,4,16
979,2009,48,Arizona,Cleveland St,Arizona,139,1112,71,1156,57,...,23,10,14,12,18,3,11,7,1,18
980,2009,48,Arizona,Utah,Arizona,137,1112,84,1428,71,...,32,9,13,16,19,16,20,7,3,22
981,2009,52,Cleveland St,Wake Forest,Cleveland St,137,1156,84,1448,69,...,13,12,17,10,23,10,18,4,13,20
982,2009,64,Morehead St,Alabama St,Morehead St,134,1287,58,1106,43,...,15,5,11,9,18,4,13,7,6,18


In [835]:
# Ok, in order to join the tables again for the losing team power
#seed, i need to rename the power seed to WPowerSeed

tourney_teams = tourney_teams.rename(columns = {'PowerSeed ': 'WPSeed'})
tourney_teams.head()

,Season,WPSeed,TeamName,LTeam,WTeam,DayNum,WTeamID,WScore,LTeamID,LScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2017,1,Villanova,Mt St Mary's,Villanova,136,1437,76,1291,56,...,21,9,12,10,17,10,9,5,5,15
1,2017,2,Kansas,Michigan St,Kansas,139,1242,90,1277,70,...,23,12,17,5,23,13,10,3,1,14
2,2017,2,Kansas,Purdue,Kansas,143,1242,98,1345,66,...,27,8,12,9,20,17,16,6,0,18
3,2017,2,Kansas,UC Davis,Kansas,137,1242,100,1413,62,...,20,16,18,9,18,7,14,5,2,19
4,2017,3,North Carolina,Arkansas,North Carolina,139,1314,72,1116,65,...,21,5,8,8,24,9,19,10,4,20


In [836]:
tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamName")
tourney_teams.head()

,Season,WPSeed,LTeam,WTeam,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2017,1,Mt St Mary's,Villanova,136,1437,76,1291,56,N,...,21,9,12,10,17,10,9,5,5,15
1,2017,2,Michigan St,Kansas,139,1242,90,1277,70,N,...,23,12,17,5,23,13,10,3,1,14
2,2017,2,Purdue,Kansas,143,1242,98,1345,66,N,...,27,8,12,9,20,17,16,6,0,18
3,2017,2,UC Davis,Kansas,137,1242,100,1413,62,N,...,20,16,18,9,18,7,14,5,2,19
4,2017,3,Arkansas,North Carolina,139,1314,72,1116,65,N,...,21,5,8,8,24,9,19,10,4,20


In [837]:
tourney_teams = pd.merge(power_seeds, tourney_teams, left_on = ["TeamName", "Season"], right_on = ["LTeam", "Season"])

In [838]:
tourney_teams.head()

tourney_teams = tourney_teams.rename(columns = {'PowerSeed ': 'LPSeed'})
tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamName")

tourney_teams.head()

,Season,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,WScore,LTeamID,LScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2017,1,32,Villanova,Wisconsin,138,1458,65,1437,62,...,16,15,21,11,19,5,8,8,1,16
1,2017,2,11,Kansas,Oregon,145,1332,74,1242,60,...,25,13,17,12,20,14,8,4,1,13
2,2017,4,3,Gonzaga,North Carolina,154,1314,71,1211,65,...,19,17,26,12,37,11,14,2,5,22
3,2017,5,41,Arizona,Xavier,143,1462,73,1112,71,...,27,10,11,14,21,14,8,4,1,18
4,2017,6,3,Kentucky,North Carolina,146,1314,75,1246,73,...,19,12,19,11,23,16,9,7,2,19


Awesone, Lets get the seed that was given, as seen in the tourney_seeds table above, the seeds come with prefixes and suffixes, what region, and if they were a play in game, an "a" or "b", we will extract these, and eonnect these to our tourney_teams data set 

In [839]:
def seed_extract(s):
    if s[-1] == 'a' or s[-1] == "b" :
        return s[1:3]
    else:
        return s[1::]
    
new = []

    
for i in tourney_seeds.Seed:
    #seed = []
    i = seed_extract(i)
    i = int(i)
    new.append(i)
    #seed.append(i)

#print(new)
    #print(seed)



In [840]:
tourney_seeds.Seed = new 

tourney_seeds.head()

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374


In [841]:
tourney_teams =  pd.merge(tourney_seeds, tourney_teams, left_on = ["TeamID", "Season"], right_on = ["WTeamID", 'Season'])

tourney_teams.tail()

,Season,Seed,TeamID,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,WScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
938,2017,4,1139,50,14,Winthrop,Butler,136,1139,76,...,25,5,10,3,24,12,9,6,1,22
939,2017,6,1153,42,22,Kansas St,Cincinnati,137,1153,75,...,18,12,15,9,14,10,7,7,2,19
940,2017,8,1116,34,30,Seton Hall,Arkansas,137,1116,77,...,17,13,21,21,25,10,15,6,4,18
941,2017,10,1455,26,38,Dayton,Wichita St,137,1455,64,...,22,15,20,10,19,11,6,6,3,23
942,2017,12,1292,18,46,Minnesota,MTSU,136,1292,81,...,21,18,21,4,20,15,8,5,3,22


In [842]:
#changing seed to WSeed
tourney_teams = tourney_teams.rename(columns = {'Seed': 'WTeamSeed'})

tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamID")

#geting LTeamSeed
tourney_teams =  pd.merge(tourney_seeds, tourney_teams, left_on = ["TeamID", "Season"], right_on = ["LTeamID", 'Season'])
tourney_teams = tourney_teams.rename(columns = {'Seed': 'LTeamSeed'})
tourney_teams = tourney_teams.drop(axis = 1, columns = "TeamID")
tourney_teams.head()

,Season,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,WScore,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,1,3,3,10,Oklahoma,Syracuse,146,1393,63,...,28,6,11,14,14,10,19,13,4,16
1,2003,2,10,6,38,Wake Forest,Auburn,139,1120,68,...,23,18,21,12,23,5,18,7,7,23
2,2003,4,12,14,46,Louisville,Butler,139,1139,79,...,25,4,4,8,19,13,14,8,2,21
3,2003,5,12,18,46,Mississippi St,Butler,137,1139,47,...,21,8,14,8,22,10,10,6,4,13
4,2003,6,3,22,10,Oklahoma St,Syracuse,139,1393,68,...,19,9,19,11,27,16,22,8,4,19


I want to see if I can add the rankings table to this data set and see if that will be useful, however I am worried that these rankings are only done for regular season games, because the tournament then reseeds and make it difficult, thus it will probably better to use tournament seed as a column and then a predictive feature. Although the tournament rarely goes chalk, 1 seeds are indeed much better than other seeds and do win more than most others, for full statistical analysis on the seeds and winning percentage, please watch this video which was helpful in our methodology aproach. https://www.youtube.com/watch?v=K_jJNfb3T04

March veterans will instinctivelly go to the 5-12 match ups when looking at upsets, in the pasy 34 years, at least one 12 has one their first round game, also the 6-11 match ups, 11 seeds actually has a winning record of 56% against the 6 seeds, and in the past 3 years the 11 has won 8 of the last 12 match ups.

However fun the upsets may be, 3 one seeds have past the round of 32 and to the "Sweet 16" 31 of the last 34 seasons, but it is very unlikely that the 1,2,3 and 4 seeds all make the second weekend in the same region, this only happens 12.5% of the time. And here inlies the beauty of March, in 32 of the last 34 years at least one team seeded 10th or worst has made the sweet 16, the trick will be determening which double diget seed to pick and which top seeds to eliminate early.

In [843]:
# reading in MASSIVE data set with rankings 

'''
This data fram contains several different ranking systems all 
of which are similar from the year 2003, these are similar 
but are different in the way they are calculated and in their historical 
predictive preformance.  We are going to use the SAG ranking,
because it is the one that predicts the best according to to our research.
'''

rankings = pd.read_csv("MasseyOrdinals.csv")

SAG_ranking = rankings.loc[rankings['SystemName'] == "SAG"]
SAG_ranking.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
3622,2003,37,SAG,1102,114
3623,2003,37,SAG,1103,193
3624,2003,37,SAG,1104,11
3625,2003,37,SAG,1105,310
3626,2003,37,SAG,1106,257


Another observation: In order to test our model, we will not have any of the tournament data like we have above, because that data will be gathered simultaneously with the games being played, so we will need to create lots of features and use regular seasaon data, from THIS season to predict THIS tournament, so we will not be able to train on past tournament data because we do not have tournament data from this season. 

Therefore we will configure our data for our regular season data in the same way that we did for our tournament data 

## Feature Engineering 

Given that we cannot use tournament data to predict tournament games, our statistical columns in our tourney_teams data will not be used.  We will create several features in order to have a data set to use for predictions 


In [844]:
testd = tourney_teams

testd = testd.drop(axis = 1, columns = ['WScore', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'])

testd.head()

,Season,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID
0,2003,1,3,3,10,Oklahoma,Syracuse,146,1393,1328
1,2003,2,10,6,38,Wake Forest,Auburn,139,1120,1448
2,2003,4,12,14,46,Louisville,Butler,139,1139,1257
3,2003,5,12,18,46,Mississippi St,Butler,137,1139,1280
4,2003,6,3,22,10,Oklahoma St,Syracuse,139,1393,1329


#### feature 1
College basketball has, with the growth of TV revenue and distribution formed into power conference over the last several years.  These "Power" conferences are the SEC, BIG 10, Big 12, ACC, Pac 10 which turned into the Pac 12 and Big East, which although not considered one in football, is still considered a basketball power conference.  This could be potentially sticky as conference allignments have changed over the years, for example, a non P-6 team, the University of Utah, went from a group of 5 conference, the Mountain West to the expanded Pac 12, back in 2011, we will explore the data below and create a feature for conference affiliation 

In [845]:
conf = pd.read_csv("TeamConferences.csv")

power_conf = ["acc", 'sec', "big_east", "big_ten", 
              "big_twelve", "pac_ten", "pac_twelve"]

p6 = []
 
for i in conf.ConfAbbrev:
    if i in power_conf:
        p6.append(1)
    else:
        p6.append(0)
        

conf['p6'] = p6

conf.head()

,Season,TeamID,ConfAbbrev,p6
0,1985,1114,a_sun,0
1,1985,1147,a_sun,0
2,1985,1204,a_sun,0
3,1985,1209,a_sun,0
4,1985,1215,a_sun,0


In [846]:
testd =  pd.merge(conf, testd, left_on = ["TeamID", "Season"], right_on = ["WTeamID", 'Season'])

testd.head()

,Season,TeamID,ConfAbbrev,p6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID
0,2003,1462,a_ten,0,14,3,55,11,Troy,Xavier,137,1462,1407
1,2003,1181,acc,1,11,3,41,9,C Michigan,Duke,138,1181,1141
2,2003,1181,acc,1,14,3,53,9,Colorado St,Duke,136,1181,1161
3,2003,1268,acc,1,3,6,11,23,Xavier,Maryland,139,1268,1462
4,2003,1268,acc,1,11,6,43,23,UNC Wilmington,Maryland,137,1268,1423


In [847]:
testd = testd.rename(columns = {'p6': 'WP6'})
testd = testd.drop(axis = 1, columns = ["ConfAbbrev", "TeamID"])
testd.head()

,Season,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID
0,2003,0,14,3,55,11,Troy,Xavier,137,1462,1407
1,2003,1,11,3,41,9,C Michigan,Duke,138,1181,1141
2,2003,1,14,3,53,9,Colorado St,Duke,136,1181,1161
3,2003,1,3,6,11,23,Xavier,Maryland,139,1268,1462
4,2003,1,11,6,43,23,UNC Wilmington,Maryland,137,1268,1423


In [848]:
testd =  pd.merge(conf, testd, left_on = ["TeamID", "Season"], right_on = ["LTeamID", 'Season'])
testd = testd.rename(columns = {'p6': 'LP6'})
testd = testd.drop(axis = 1, columns = ["ConfAbbrev", "TeamID"])
testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181


### feature 2: 
32 of the 64 spots available in the tournament go to conference tournament winners, many of these conferences are not P6 and are "one bid leagues", however some are not.  Teams that do not win their conferences are get into the tournament with "at large bids". We will create a feature to find teams that won their conference titles, and recieved an automatic bid. A 1 will denote confernce tournament champion, else 0 
---- caviat the Ivy league sends their regular season not their conference champion, we will investigate the data to see if that is consistant with our data. 

In [849]:
conf_tour = pd.read_csv("ConferenceTourneyGames.csv")
conf_tour.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,121,1194,1144
1,2001,a_sun,121,1416,1240
2,2001,a_sun,122,1209,1194
3,2001,a_sun,122,1359,1239
4,2001,a_sun,122,1391,1273


In [850]:
'''
Many of the tournament titles are played on "Selection Sunday"
the day the tournament is seeded.  others are played throughout
"Championship Week". We will need to find a way to group by 
conference and select the winner on the last day a game was played 
for a particular conference 

'''

conf_champ = conf_tour.groupby(["Season", "ConfAbbrev"])["DayNum"].max() 

conf_champ = pd.DataFrame(conf_champ)
conf_champ.head()



DayNum
Season ConfAbbrev        
2001   a_sun          124
       a_ten          131
       acc            132
       aec            131
       big_east       131

In [851]:
#awesome going to try and merge tables to then get back to WTeamID 

conf_champ = pd.merge(conf_champ, conf_tour, left_on = ["Season", "ConfAbbrev", "DayNum"], right_on = ["Season", "ConfAbbrev", "DayNum"])

conf_champ.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,124,1209,1407
1,2001,a_ten,131,1396,1269
2,2001,acc,132,1181,1314
3,2001,aec,131,1220,1174
4,2001,big_east,131,1130,1338


In [852]:
conf_champ["ConfChamp"] = 1

conf_champ.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID,ConfChamp
0,2001,a_sun,124,1209,1407,1
1,2001,a_ten,131,1396,1269,1
2,2001,acc,132,1181,1314,1
3,2001,aec,131,1220,1174,1
4,2001,big_east,131,1130,1338,1


In [853]:
testd.shape

(943, 12)

In [854]:
testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181


In [855]:
testd = pd.merge(testd, conf_champ, on = ["Season", "WTeamID"], how = "left")


In [856]:
testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum_x,WTeamID,LTeamID_x,ConfAbbrev,DayNum_y,LTeamID_y,ConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,NaN,NaN,NaN,NaN
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,wac,131.0,1305.0,1.0
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,NaN,NaN,NaN,NaN
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,NaN,NaN,NaN,NaN
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,NaN,NaN,NaN,NaN


In [857]:
testd.shape

(943, 16)

In [858]:
#perfect, I did a left join on that to get the team and if they won their conference champion 
# I will drop these columns then repeat for my LTeamID

testd = testd.drop(axis = 1, columns = ["ConfAbbrev", "DayNum_y", "LTeamID_y"])

testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum_x,WTeamID,LTeamID_x,ConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,NaN
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,1.0
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,NaN
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,NaN
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,NaN


In [859]:
# filling in null data with 0s instead of NA 

testd.loc[testd["ConfChamp"].isnull(), "ConfChamp"] = 0


testd = testd.rename(columns = {'ConfChamp': 'WConfChamp', "LTeamID_x": "LTeamID"})


testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum_x,WTeamID,LTeamID,WConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,0.0
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,1.0
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,0.0
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,0.0
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,0.0


In [860]:
# ok doing the same for my losing teams 
testd = pd.merge(testd, conf_champ, on = ["Season", "LTeamID"], how = "left")

testd.head()


,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum_x,WTeamID_x,LTeamID,WConfChamp,ConfAbbrev,DayNum,WTeamID_y,ConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,0.0,NaN,NaN,NaN,NaN
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,1.0,NaN,NaN,NaN,NaN
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,0.0,NaN,NaN,NaN,NaN
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,0.0,NaN,NaN,NaN,NaN
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,0.0,NaN,NaN,NaN,NaN


In [862]:
#cleaning up columns 
testd = testd.drop(axis = 1, columns = ["ConfAbbrev", "DayNum", "WTeamID_y"])
testd.head()


,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum_x,WTeamID_x,LTeamID,WConfChamp,ConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,0.0,NaN
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,1.0,NaN
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,0.0,NaN
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,0.0,NaN
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,0.0,NaN


In [863]:
testd.loc[testd["ConfChamp"].isnull(), "ConfChamp"] = 0

testd = testd.rename(columns = {'ConfChamp': 'LConfChamp', "WTeamID_x": "WTeamID", "DayNum_x": "DayNum"})


testd.head()

,Season,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WTeamID,LTeamID,WConfChamp,LConfChamp
0,2003,0,0,14,3,55,11,Troy,Xavier,137,1462,1407,0.0,0.0
1,2003,0,0,4,13,16,52,Dayton,Tulsa,136,1409,1173,1.0,0.0
2,2003,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1120,1386,0.0,0.0
3,2003,0,1,3,6,11,23,Xavier,Maryland,139,1268,1462,0.0,0.0
4,2003,1,1,3,2,9,5,Duke,Kansas,143,1242,1181,0.0,0.0


## Third group of Features 
These will all be in the same vain.  These are going to be regular season averages for several statistical categories, field goal percentage, three point percentage, thre pointers made, steals, defensive rebounds, offensive rebounds, free throw percentage, average points per game, average blocks, average turnovers, point differential, opponents 3p percentage, wins, losses.  We know that the more features we create, the more prone we will be to over fitting, however, we feel that these will be key indicators, and that if we do end up over fitting our model, we can always features out. 

In [864]:
reg_deets.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [865]:
reg_deets["WSeasonfg%"] = reg_deets["WFGM"] / reg_deets["WFGA"] 
reg_deets["WSeasonFT%"] = reg_deets["WFTM"] / reg_deets["WFTA"]
reg_deets["WSeason3%"] = reg_deets["WFGM3"] / reg_deets["WFGA3"]

reg_deets["LSeasonfg%"] = reg_deets["LFGM"] / reg_deets["LFGA"] 
reg_deets["LSeasonFT%"] = reg_deets["LFTM"] / reg_deets["LFTA"]
reg_deets["LSeason3%"] = reg_deets["LFGM3"] / reg_deets["LFGA3"]


In [866]:
reg_avgs_WFGP = reg_deets.groupby(["Season", "WTeamID"])["WSeasonfg%"].mean() 
reg_avgs_WFGP
reg_avgs_WFGP = pd.DataFrame(reg_avgs_WFGP)
reg_avgs_WFGP.head()

WSeasonfg%
Season WTeamID            
2003   1102       0.567934
       1103       0.542563
       1104       0.444393
       1105       0.413500
       1106       0.461775

In [867]:
testd = pd.merge(reg_avgs_WFGP, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()


,WTeamID,Season,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,LTeamID,WConfChamp,LConfChamp
0,1112,2003,0.473342,0,1,16,1,61,4,Vermont,Arizona,136,1436,0.0,0.0
1,1112,2003,0.473342,1,1,5,1,17,4,Notre Dame,Arizona,143,1323,0.0,0.0
2,1112,2003,0.473342,0,1,9,1,33,4,Gonzaga,Arizona,138,1211,0.0,1.0
3,1113,2003,0.522554,0,1,7,10,25,37,Memphis,Arizona St,136,1272,0.0,0.0
4,1120,2003,0.500417,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1386,0.0,0.0


In [868]:
reg_avgs_WFGP = reg_deets.groupby(["Season", "WTeamID"])["WSeasonFT%"].mean() 
reg_avgs_WFGP
reg_avgs_WFTP = pd.DataFrame(reg_avgs_WFTP)
reg_avgs_WFTP.head()

WSeasonFT%
Season WTeamID            
2003   1102       0.631688
       1103       0.722603
       1104       0.709384
       1105       0.743782
       1106       0.591138

In [869]:
testd = pd.merge(reg_avgs_WFTP, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()

,WTeamID,Season,WSeasonFT%,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,LTeamID,WConfChamp,LConfChamp
0,1112,2003,0.703983,0.473342,0,1,16,1,61,4,Vermont,Arizona,136,1436,0.0,0.0
1,1112,2003,0.703983,0.473342,1,1,5,1,17,4,Notre Dame,Arizona,143,1323,0.0,0.0
2,1112,2003,0.703983,0.473342,0,1,9,1,33,4,Gonzaga,Arizona,138,1211,0.0,1.0
3,1113,2003,0.693383,0.522554,0,1,7,10,25,37,Memphis,Arizona St,136,1272,0.0,0.0
4,1120,2003,0.660255,0.500417,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1386,0.0,0.0


In [870]:
reg_avgs_W3P = reg_deets.groupby(["Season", "WTeamID"])["WSeason3%"].mean() 
reg_avgs_W3P
reg_avgs_W3P = pd.DataFrame(reg_avgs_W3P)
reg_avgs_W3P.head()

WSeason3%
Season WTeamID           
2003   1102      0.449882
       1103      0.362828
       1104      0.347418
       1105      0.391674
       1106      0.383482

In [871]:
testd = pd.merge(reg_avgs_W3P, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()

,WTeamID,Season,WSeason3%,WSeasonFT%,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,LTeamID,WConfChamp,LConfChamp
0,1112,2003,0.362485,0.703983,0.473342,0,1,16,1,61,4,Vermont,Arizona,136,1436,0.0,0.0
1,1112,2003,0.362485,0.703983,0.473342,1,1,5,1,17,4,Notre Dame,Arizona,143,1323,0.0,0.0
2,1112,2003,0.362485,0.703983,0.473342,0,1,9,1,33,4,Gonzaga,Arizona,138,1211,0.0,1.0
3,1113,2003,0.331964,0.693383,0.522554,0,1,7,10,25,37,Memphis,Arizona St,136,1272,0.0,0.0
4,1120,2003,0.396498,0.660255,0.500417,0,1,7,10,26,38,St Joseph's PA,Auburn,137,1386,0.0,0.0


In [872]:
reg_avgs_LFGP = reg_deets.groupby(["Season", "LTeamID"])["LSeasonfg%"].mean() 
reg_avgs_LFGP
reg_avgs_LFGP = pd.DataFrame(reg_avgs_LFGP)
reg_avgs_LFGP.head()

LSeasonfg%
Season LTeamID            
2003   1102       0.424810
       1103       0.435972
       1104       0.381478
       1105       0.389832
       1106       0.394117

In [873]:
testd = pd.merge(reg_avgs_LFGP, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

testd.head()

,LTeamID,Season,LSeasonfg%,WTeamID,WSeason3%,WSeasonFT%,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1104,2003,0.381478,1231,0.367781,0.717359,0.444165,1,1,10,7,40,28,Alabama,Indiana,137,0.0,0.0
1,1112,2003,0.382065,1242,0.361208,0.668733,0.519173,1,1,1,2,4,5,Arizona,Kansas,145,0.0,0.0
2,1113,2003,0.414795,1242,0.361208,0.668733,0.519173,1,1,10,2,37,5,Arizona St,Kansas,138,0.0,0.0
3,1120,2003,0.417612,1393,0.374792,0.701951,0.492933,1,1,10,3,38,10,Auburn,Syracuse,144,0.0,0.0
4,1122,2003,0.377235,1257,0.368644,0.701819,0.472277,0,0,13,4,50,14,Austin Peay,Louisville,137,1.0,0.0


In [874]:
reg_avgs_LFTP = reg_deets.groupby(["Season", "LTeamID"])["LSeasonFT%"].mean() 
reg_avgs_LFTP
reg_avgs_LFTP = pd.DataFrame(reg_avgs_LFTP)
reg_avgs_LFTP.head()

LSeasonFT%
Season LTeamID            
2003   1102       0.650437
       1103       0.747034
       1104       0.698651
       1105       0.697004
       1106       0.650908

In [875]:
testd = pd.merge(reg_avgs_LFTP, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

testd.head()

,LTeamID,Season,LSeasonFT%,LSeasonfg%,WTeamID,WSeason3%,WSeasonFT%,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1104,2003,0.698651,0.381478,1231,0.367781,0.717359,0.444165,1,1,10,7,40,28,Alabama,Indiana,137,0.0,0.0
1,1112,2003,0.677580,0.382065,1242,0.361208,0.668733,0.519173,1,1,1,2,4,5,Arizona,Kansas,145,0.0,0.0
2,1113,2003,0.646678,0.414795,1242,0.361208,0.668733,0.519173,1,1,10,2,37,5,Arizona St,Kansas,138,0.0,0.0
3,1120,2003,0.639548,0.417612,1393,0.374792,0.701951,0.492933,1,1,10,3,38,10,Auburn,Syracuse,144,0.0,0.0
4,1122,2003,0.685317,0.377235,1257,0.368644,0.701819,0.472277,0,0,13,4,50,14,Austin Peay,Louisville,137,1.0,0.0


In [876]:
reg_avgs_L3P = reg_deets.groupby(["Season", "LTeamID"])["LSeason3%"].mean() 
reg_avgs_L3P
reg_avgs_L3P = pd.DataFrame(reg_avgs_L3P)
testd = pd.merge(reg_avgs_L3P, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

testd.head()

,LTeamID,Season,LSeason3%,LSeasonFT%,LSeasonfg%,WTeamID,WSeason3%,WSeasonFT%,WSeasonfg%,LP6,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1104,2003,0.291478,0.698651,0.381478,1231,0.367781,0.717359,0.444165,1,1,10,7,40,28,Alabama,Indiana,137,0.0,0.0
1,1112,2003,0.255856,0.677580,0.382065,1242,0.361208,0.668733,0.519173,1,1,1,2,4,5,Arizona,Kansas,145,0.0,0.0
2,1113,2003,0.322505,0.646678,0.414795,1242,0.361208,0.668733,0.519173,1,1,10,2,37,5,Arizona St,Kansas,138,0.0,0.0
3,1120,2003,0.285127,0.639548,0.417612,1393,0.374792,0.701951,0.492933,1,1,10,3,38,10,Auburn,Syracuse,144,0.0,0.0
4,1122,2003,0.325680,0.685317,0.377235,1257,0.368644,0.701819,0.472277,0,0,13,4,50,14,Austin Peay,Louisville,137,1.0,0.0


In [877]:
print(reg_deets.columns)

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'WSeasonfg%', 'WSeasonFT%', 'WSeason3%', 'LSeasonfg%', 'LSeasonFT%',
       'LSeason3%'],
      dtype='object')


In [878]:
# Next set are means of rebounds, steals, blocks etc 

#first defensive rebounds 

reg_avgs_WDR = reg_deets.groupby(["Season", "WTeamID"])["WDR"].mean() 
reg_avgs_WDR = pd.DataFrame(reg_avgs_WDR)
reg_avgs_WDR.head()

WDR
Season WTeamID           
2003   1102     19.333333
       1103     21.461538
       1104     26.411765
       1105     25.857143
       1106     28.000000

In [879]:
testd = pd.merge(reg_avgs_WDR, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()

,WTeamID,Season,WDR,LTeamID,LSeason3%,LSeasonFT%,LSeasonfg%,WSeason3%,WSeasonFT%,WSeasonfg%,...,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1112,2003,28.160000,1211,0.346468,0.740122,0.438653,0.362485,0.703983,0.473342,...,1,9,1,33,4,Gonzaga,Arizona,138,0.0,1.0
1,1112,2003,28.160000,1323,0.356099,0.713305,0.410541,0.362485,0.703983,0.473342,...,1,5,1,17,4,Notre Dame,Arizona,143,0.0,0.0
2,1112,2003,28.160000,1436,0.325834,0.609571,0.398273,0.362485,0.703983,0.473342,...,1,16,1,61,4,Vermont,Arizona,136,0.0,0.0
3,1113,2003,25.222222,1272,0.306028,0.646861,0.401633,0.331964,0.693383,0.522554,...,1,7,10,25,37,Memphis,Arizona St,136,0.0,0.0
4,1120,2003,25.052632,1386,0.301701,0.601391,0.410140,0.396498,0.660255,0.500417,...,1,7,10,26,38,St Joseph's PA,Auburn,137,0.0,0.0


In [880]:
#offensive rebounds

reg_avgs_WOR = reg_deets.groupby(["Season", "WTeamID"])["WOR"].mean() 
reg_avgs_WOR = pd.DataFrame(reg_avgs_WOR)


reg_avgs_WOR.head()

testd = pd.merge(reg_avgs_WOR, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()

,WTeamID,Season,WOR,WDR,LTeamID,LSeason3%,LSeasonFT%,LSeasonfg%,WSeason3%,WSeasonFT%,...,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1112,2003,14.840000,28.160000,1211,0.346468,0.740122,0.438653,0.362485,0.703983,...,1,9,1,33,4,Gonzaga,Arizona,138,0.0,1.0
1,1112,2003,14.840000,28.160000,1323,0.356099,0.713305,0.410541,0.362485,0.703983,...,1,5,1,17,4,Notre Dame,Arizona,143,0.0,0.0
2,1112,2003,14.840000,28.160000,1436,0.325834,0.609571,0.398273,0.362485,0.703983,...,1,16,1,61,4,Vermont,Arizona,136,0.0,0.0
3,1113,2003,14.000000,25.222222,1272,0.306028,0.646861,0.401633,0.331964,0.693383,...,1,7,10,25,37,Memphis,Arizona St,136,0.0,0.0
4,1120,2003,12.210526,25.052632,1386,0.301701,0.601391,0.410140,0.396498,0.660255,...,1,7,10,26,38,St Joseph's PA,Auburn,137,0.0,0.0


In [881]:
# average steals per season for winning team 

reg_avgs_WStl = reg_deets.groupby(["Season", "WTeamID"])["WStl"].mean() 
reg_avgs_WStl = pd.DataFrame(reg_avgs_WStl)


reg_avgs_WStl.head()

testd = pd.merge(reg_avgs_WStl, testd, left_on = ["WTeamID", "Season"], right_on = ["WTeamID", "Season"])

testd.head()

,WTeamID,Season,WStl,WOR,WDR,LTeamID,LSeason3%,LSeasonFT%,LSeasonfg%,WSeason3%,...,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1112,2003,8.440000,14.840000,28.160000,1211,0.346468,0.740122,0.438653,0.362485,...,1,9,1,33,4,Gonzaga,Arizona,138,0.0,1.0
1,1112,2003,8.440000,14.840000,28.160000,1323,0.356099,0.713305,0.410541,0.362485,...,1,5,1,17,4,Notre Dame,Arizona,143,0.0,0.0
2,1112,2003,8.440000,14.840000,28.160000,1436,0.325834,0.609571,0.398273,0.362485,...,1,16,1,61,4,Vermont,Arizona,136,0.0,0.0
3,1113,2003,5.722222,14.000000,25.222222,1272,0.306028,0.646861,0.401633,0.331964,...,1,7,10,25,37,Memphis,Arizona St,136,0.0,0.0
4,1120,2003,8.947368,12.210526,25.052632,1386,0.301701,0.601391,0.410140,0.396498,...,1,7,10,26,38,St Joseph's PA,Auburn,137,0.0,0.0


In [882]:
# average steals per season for Lteam 

#Assists
reg_avgs_LAst = reg_deets.groupby(["Season", "LTeamID"])["LAst"].mean() 
reg_avgs_LAst = pd.DataFrame(reg_avgs_LAst)
testd = pd.merge(reg_avgs_LAst, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

#Steals
reg_avgs_LStl = reg_deets.groupby(["Season", "LTeamID"])["LStl"].mean() 
reg_avgs_LStl = pd.DataFrame(reg_avgs_LStl)
testd = pd.merge(reg_avgs_LStl, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

#LOR
reg_avgs_LOR = reg_deets.groupby(["Season", "LTeamID"])["LOR"].mean() 
reg_avgs_LOR = pd.DataFrame(reg_avgs_LOR)
testd = pd.merge(reg_avgs_LOR, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])

#LDR
reg_avgs_LDR = reg_deets.groupby(["Season", "LTeamID"])["LDR"].mean() 
reg_avgs_LDR = pd.DataFrame(reg_avgs_LDR)
testd = pd.merge(reg_avgs_LDR, testd, left_on = ["LTeamID", "Season"], right_on = ["LTeamID", "Season"])


testd.head()

,LTeamID,Season,LDR,LOR,LStl,LAst,WTeamID,WStl,WOR,WDR,...,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp
0,1104,2003,20.090909,13.636364,5.636364,9.181818,1231,5.000000,11.850000,28.150000,...,1,10,7,40,28,Alabama,Indiana,137,0.0,0.0
1,1112,2003,23.333333,18.000000,8.666667,13.000000,1242,11.130435,13.869565,27.478261,...,1,1,2,4,5,Arizona,Kansas,145,0.0,0.0
2,1113,2003,20.181818,13.181818,4.363636,13.272727,1242,11.130435,13.869565,27.478261,...,1,10,2,37,5,Arizona St,Kansas,138,0.0,0.0
3,1120,2003,18.272727,11.454545,6.454545,9.909091,1393,8.625000,13.333333,27.708333,...,1,10,3,38,10,Auburn,Syracuse,144,0.0,0.0
4,1122,2003,21.571429,11.285714,7.285714,11.857143,1257,7.583333,13.083333,25.583333,...,0,13,4,50,14,Austin Peay,Louisville,137,1.0,0.0


In [894]:
# I want to be able to find their season averages for defense and get the record for each team

reg_WT_wins = reg_deets.groupby(["Season", "WTeamID"])["WTeamID"].count() 
reg_WT_wins = pd.DataFrame(reg_WT_wins)
reg_WT_wins.head()

WTeamID
Season WTeamID         
2003   1102          12
       1103          13
       1104          17
       1105           7
       1106          13

In [893]:
reg_WT_wins = reg_WT_wins.rename(columns = {'WTeam': 'LTeam', 'TeamName_y':'WTeam'})


C:\Users\13855\Anaconda_Re_do\lib\site-packages\IPython\core\interactiveshell.py:3267: FutureWarning: 'WTeamID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,WTeamID,Season,LTeamID,LDR,LOR,LStl,LAst,WStl,WOR,WDR,...,WP6,LTeamSeed,WTeamSeed,LPSeed,WPSeed,LTeam,WTeam,DayNum,WConfChamp,LConfChamp


In [884]:
print(reg_deets.columns)

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'WSeasonfg%', 'WSeasonFT%', 'WSeason3%', 'LSeasonfg%', 'LSeasonFT%',
       'LSeason3%'],
      dtype='object')


In [560]:
# favwin = []
# for i in tourney_teams[0]:
#     if toureny_teams.WPSeed[i] < tourney_teams.LPSeed[i]:
#         favwin.append(1)
#     else:
#         favwin.append(0)

# favwin

KeyError: 0